In [1]:
import pandas as pd
drug = pd.read_csv("../data/eICU/infusiondrug.csv")
diagnosis = pd.read_csv("../data/eICU/diagnosis.csv")
patients_weight = pd.read_csv("../data/eICU/patient.csv")[["patientunitstayid", "admissionweight"]]
drug = drug.merge(patients_weight, on="patientunitstayid")

C:\Users\eythe\AppData\Local\Temp\ipykernel_7108\1321175797.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  drug = pd.read_csv("../data/eICU/infusiondrug.csv")


In [2]:
patient = pd.read_csv("../data/eICU/patient.csv")

In [3]:
# add hospitalID and unitType and wardID to the drug table corelated to patientunitstayid
drug["hospitalid"] = drug["patientunitstayid"].map(patient.set_index("patientunitstayid")["hospitalid"])
drug["unittype"] = drug["patientunitstayid"].map(patient.set_index("patientunitstayid")["unittype"])
drug["wardid"] = drug["patientunitstayid"].map(patient.set_index("patientunitstayid")["wardid"])



In [4]:
# filter drug from NA in drugname, patientunitstayid, hospitalid, unittype, wardid
drug = drug[~drug['drugname'].isna()]
drug = drug[~drug['patientunitstayid'].isna()]
drug = drug[~drug['hospitalid'].isna()]
drug = drug[~drug['unittype'].isna()]
drug = drug[~drug['wardid'].isna()]

#filter drug to include only drugnames that start with "Norepinephrine"
drug = drug[drug['drugname'].str.startswith("Norepinephrine")]

In [5]:
# sort drug by patientunitstayid and drugstartoffset
drug = drug.sort_values(by=["patientunitstayid", "infusionoffset"])
drug

,infusiondrugid,patientunitstayid,infusionoffset,drugname,drugrate,infusionrate,drugamount,volumeoffluid,patientweight,admissionweight,hospitalid,unittype,wardid
4,1969910,242082,55,Norepinephrine (mcg/min),2.13,8.0,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
2,1968206,242082,125,Norepinephrine (mcg/min),7,26.3,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
10,1985898,242082,230,Norepinephrine (mcg/min),9.07,34.0,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
12,1972141,242082,275,Norepinephrine (mcg/min),10,37.5,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
1,1998443,242082,425,Norepinephrine (mcg/min),10.93,41.0,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4803535,75122092,3353251,3823,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,15,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104
4803681,75308975,3353251,3968,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,16,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104
4803654,75246300,3353251,4043,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,12,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104
4803592,75148203,3353251,4358,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,10,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104


In [6]:
# plot histogram of number of differnt patientsunitstayid for every hospitalid for each unittype and total
drug.groupby(["hospitalid", "unittype"])["patientunitstayid"].nunique().sort_values()
# drug.groupby("hospitalid")["patientunitstayid"].nunique().sort_values()

hospitalid  unittype    
342         Med-Surg ICU      1
151         Med-Surg ICU      1
265         MICU              1
156         Med-Surg ICU      1
438         Med-Surg ICU      1
                           ... 
157         Med-Surg ICU    375
167         Med-Surg ICU    384
188         MICU            430
300         Med-Surg ICU    452
199         Med-Surg ICU    525
Name: patientunitstayid, Length: 195, dtype: int64

In [55]:
# for each hospitalid, give me the number of patientsunitstayid for each unittype and their sum
hospis = drug.groupby(["hospitalid", "unittype"])["patientunitstayid"].nunique().sort_values().unstack().fillna(0).assign(sum=lambda x: x.sum(axis=1)).sort_values(by="sum", ascending=False)



In [61]:
hospis[hospis['sum'] > 250].size


207

In [8]:
bp = pd.read_csv("../preprocess/filtered_bp_eicu.csv")

In [10]:
# sort bp by stay_id and cur_bp_time
bp = bp.sort_values(by=["stay_id", "cur_bp_time"])
bp

,Unnamed: 0,stay_id,cur_bp_time,cur_bp
92,161,141227,597,92.0
105,191,141227,602,92.0
136,263,141227,607,88.0
120,227,141227,612,86.0
141,275,141227,617,95.0
...,...,...,...,...
3328428,950941,3353251,12179,65.0
3328499,951041,3353251,12184,65.0
3328497,951037,3353251,12189,64.0
3328502,951045,3353251,12194,60.0


In [11]:
# filter bp for patients that have norepinephrine
bp = bp[bp["stay_id"].isin(drug["patientunitstayid"])]
bp

,Unnamed: 0,stay_id,cur_bp_time,cur_bp
435480,1494922,243334,51,199.0
435501,1494950,243334,56,199.0
435504,1494954,243334,61,199.0
435470,1494906,243334,66,199.0
435474,1494910,243334,71,199.0
...,...,...,...,...
3328428,950941,3353251,12179,65.0
3328499,951041,3353251,12184,65.0
3328497,951037,3353251,12189,64.0
3328502,951045,3353251,12194,60.0


3439

In [12]:
# add column of interval between current bp time and next bp time for each patient
bp["next_bp_time"] = bp.groupby("stay_id")["cur_bp_time"].shift(-1)
bp["interval"] = bp["next_bp_time"] - bp["cur_bp_time"]
bp

C:\Users\eythe\AppData\Local\Temp\ipykernel_7108\2858970653.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bp["next_bp_time"] = bp.groupby("stay_id")["cur_bp_time"].shift(-1)
C:\Users\eythe\AppData\Local\Temp\ipykernel_7108\2858970653.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bp["interval"] = bp["next_bp_time"] - bp["cur_bp_time"]


,Unnamed: 0,stay_id,cur_bp_time,cur_bp,next_bp_time,interval
435480,1494922,243334,51,199.0,56.0,5.0
435501,1494950,243334,56,199.0,61.0,5.0
435504,1494954,243334,61,199.0,66.0,5.0
435470,1494906,243334,66,199.0,71.0,5.0
435474,1494910,243334,71,199.0,76.0,5.0
...,...,...,...,...,...,...
3328428,950941,3353251,12179,65.0,12184.0,5.0
3328499,951041,3353251,12184,65.0,12189.0,5.0
3328497,951037,3353251,12189,64.0,12194.0,5.0
3328502,951045,3353251,12194,60.0,12199.0,5.0


In [13]:
# add column of hospitalid, unittype and wardid to the bp table corelated to patientunitstayid
bp["hospitalid"] = bp["stay_id"].map(patient.set_index("patientunitstayid")["hospitalid"])
bp["unittype"] = bp["stay_id"].map(patient.set_index("patientunitstayid")["unittype"])
bp["wardid"] = bp["stay_id"].map(patient.set_index("patientunitstayid")["wardid"])
bp

C:\Users\eythe\AppData\Local\Temp\ipykernel_7108\4253266711.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bp["hospitalid"] = bp["stay_id"].map(patient.set_index("patientunitstayid")["hospitalid"])
C:\Users\eythe\AppData\Local\Temp\ipykernel_7108\4253266711.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bp["unittype"] = bp["stay_id"].map(patient.set_index("patientunitstayid")["unittype"])
C:\Users\eythe\AppData\Local\Temp\ipykernel_7108\4253266711.py:4: SettingWithCopyWarning: 
A value is trying t

,Unnamed: 0,stay_id,cur_bp_time,cur_bp,next_bp_time,interval,hospitalid,unittype,wardid
435480,1494922,243334,51,199.0,56.0,5.0,79,Med-Surg ICU,133
435501,1494950,243334,56,199.0,61.0,5.0,79,Med-Surg ICU,133
435504,1494954,243334,61,199.0,66.0,5.0,79,Med-Surg ICU,133
435470,1494906,243334,66,199.0,71.0,5.0,79,Med-Surg ICU,133
435474,1494910,243334,71,199.0,76.0,5.0,79,Med-Surg ICU,133
...,...,...,...,...,...,...,...,...,...
3328428,950941,3353251,12179,65.0,12184.0,5.0,458,Cardiac ICU,1104
3328499,951041,3353251,12184,65.0,12189.0,5.0,458,Cardiac ICU,1104
3328497,951037,3353251,12189,64.0,12194.0,5.0,458,Cardiac ICU,1104
3328502,951045,3353251,12194,60.0,12199.0,5.0,458,Cardiac ICU,1104


In [14]:
# bp_big will consist of patients from hospitalid with more then 400 differnt patientsunitstayid
bp_big = bp[bp["hospitalid"].isin(drug.groupby("hospitalid")["patientunitstayid"].nunique().sort_values()[drug.groupby("hospitalid")["patientunitstayid"].nunique().sort_values() > 400].index)]
bp_big

,Unnamed: 0,stay_id,cur_bp_time,cur_bp,next_bp_time,interval,hospitalid,unittype,wardid
821904,1679952,426975,1051,63.0,1056.0,5.0,142,Med-Surg ICU,256
821898,1679936,426975,1056,62.0,1061.0,5.0,142,Med-Surg ICU,256
821888,1679920,426975,1061,64.0,1066.0,5.0,142,Med-Surg ICU,256
822021,1680165,426975,1066,71.0,1071.0,5.0,142,Med-Surg ICU,256
822016,1680149,426975,1071,77.0,1076.0,5.0,142,Med-Surg ICU,256
...,...,...,...,...,...,...,...,...,...
2656663,672834,3130750,4540,199.0,4545.0,5.0,420,SICU,1027
2656622,672741,3130750,4545,199.0,4550.0,5.0,420,SICU,1027
2656630,672757,3130750,4550,199.0,4555.0,5.0,420,SICU,1027
2656041,670562,3130750,4555,199.0,4560.0,5.0,420,SICU,1027


In [15]:
# num of patients in bp_big
bp_big["stay_id"].nunique()

1637

In [49]:
# give me a list of 5 random patients with more than 0 occurences from bp_big names pats
pats = bp_big["stay_id"].value_counts()[bp_big["stay_id"].value_counts() > 0].sample(5)

In [54]:
# create a plot of bp over time for each patient using plotly subplot and add to each patient plot the norepinephrine infusion from drug table as scatter plot
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np
for i in range(10):
    # assign random patient to pat
    pat = bp_big["stay_id"].value_counts()[bp_big["stay_id"].value_counts() > 0].sample(5).index[0]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=bp_big[bp_big["stay_id"] == pat]["cur_bp_time"], y=bp_big[bp_big["stay_id"] == pat]["cur_bp"], name="MAP"))
    # scatter plot of norepinephrine infusion, not line plot
    # fig.add_trace(go.Scatter(x=drug[drug["patientunitstayid"] == pat]["infusionoffset"], y=drug[drug["patientunitstayid"] == pat]["drugrate"], name="drug", mode="markers", marker=dict(color="red")), row=i+1, col=1)
    x = drug[drug["patientunitstayid"] == pat]["infusionoffset"]
    fig.add_trace(go.Scatter(x=x, y=65 * np.ones(x.size), name="NOR", mode="markers", marker=dict(color="red")))
    fig.update_layout(title_text="bp over time for 5 patients")
    fig.write_image(f"bp{i}.png")



In [19]:
# create histograms of bp for each patient
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
fig = make_subplots(rows=5, cols=1, subplot_titles=[str(pat) for pat in pats.index])
for i, pat in enumerate(pats.index):
    fig.add_trace(go.Histogram(x=bp_big[bp_big["stay_id"] == pat]["cur_bp"], name="bp"), row=i+1, col=1)
fig.update_layout(height=1000, width=1000, title_text="bp over time for 5 patients")
fig.write_html("bp_hist.html")
fig.show()

In [28]:
# if bp is a guassian process, with some kernel on the time and druginfusionoffset, we can use the kernel to predict the next bp
# create a learner
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RationalQuadratic, ExpSineSquared
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import PairwiseKernel
from sklearn.gaussian_process.kernels import StationaryKernelMixin, NormalizedKernelMixin, Kernel
from sklearn.gaussian_process.kernels import Hyperparameter
# learn a model for each patient
models = [GaussianProcessRegressor(kernel=C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2)), alpha=0.1, n_restarts_optimizer=9, normalize_y=True) for pat in pats.index]
# make plot of predictions for each patient
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
fig = make_subplots(rows=5, cols=1, subplot_titles=[str(pat) for pat in pats.index])
for i, pat in enumerate(pats.index):
    # get the data for the patient
    x = pd.DataFrame({'time': bp_big[bp_big["stay_id"] == pat]["cur_bp_time"].to_numpy(), 'drug': bp_big[bp_big["stay_id"] == pat]["cur_bp_time"].to_numpy()})
    # For each bp, in a new column, put the drugAmount of the drug that was given at 5 minutes before the bp, and 0 otherwise
    x['drug'] = x['drug'].apply(lambda t: drug[(drug["patientunitstayid"] == pat) & (drug["infusionoffset"] <= t) & (drug["infusionoffset"] >= t - 5)]["drugamount"].to_numpy()[0] if drug[(drug["patientunitstayid"] == pat) & (drug["infusionoffset"] <= t) & (drug["infusionoffset"] >= t - 5)]["drugamount"].to_numpy().size > 0 else 0)
    y = bp_big[bp_big["stay_id"] == pat]["cur_bp"].to_numpy()
    #clean the data from Nan, and inf
    x = x.replace([np.inf, -np.inf], np.nan).dropna()
    y = y[x.index]

    # find first and last drug infusion
    time_of_first_drug = drug[drug["patientunitstayid"] == pat]["infusionoffset"].min()
    time_of_last_drug = drug[drug["patientunitstayid"] == pat]["infusionoffset"].max()
    # remove data that is 30 minutes before the first drug infusion, and 30 minutes after the last drug infusion
    k = (x['time'] >= time_of_first_drug - 30) & (x['time'] <= time_of_last_drug + 30)
    x = x[k]
    y = y[k]



    # split to train and test, when the test is the 10% of the latest data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=False)
    # plot the data
    fig.add_trace(go.Scatter(x=x_train['time'], y=y_train, name="bp"), row=i+1, col=1)

    # learn the model
    models[i].fit(x_train, y_train)
    # predict the test data
    y_pred = models[i].predict(x_test)
    # plot the test data predictions
    fig.add_trace(go.Scatter(x=x_test['time'], y=y_pred, name="bp_pred"), row=i+1, col=1)
    # plot the actual test data
    fig.add_trace(go.Scatter(x=x_test['time'], y=y_test, name="bp_test"), row=i+1, col=1)
    #plot the predictions for the train data as well in a different color
    y_train_pred = models[i].predict(x_train)
    fig.add_trace(go.Scatter(x=x_train['time'], y=y_train_pred, name="bp_pred_train", line=dict(color="red")), row=i+1, col=1)

    # add mse and corr for train and test to the plot
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max(), text="mse_train: " + str(mean_squared_error(y_train, y_train_pred)), showarrow=False, row=i+1, col=1)
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max() - 5, text="corr_train: " + str(np.corrcoef(y_train, y_train_pred)[0, 1]), showarrow=False, row=i+1, col=1)
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max() - 10, text="mse_test: " + str(mean_squared_error(y_test, y_pred)), showarrow=False, row=i+1, col=1)
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max() - 15, text="corr_test: " + str(np.corrcoef(y_test, y_pred)[0, 1]), showarrow=False, row=i+1, col=1)
    # add drug markers
    x = drug[drug["patientunitstayid"] == pat]["infusionoffset"]
    fig.add_trace(go.Scatter(x=x, y=65 * np.ones(x.size), name="drug", mode="markers", marker=dict(color="red")), row=i+1, col=1)

    # add predictions for learner fitted only on time and not on drug
    models[i].kernel = RBF(10, (1e-2, 1e2))
    models[i].fit(x_train[['time']], y_train)
    y_pred = models[i].predict(x_test[['time']])
    fig.add_trace(go.Scatter(x=x_test['time'], y=y_pred, name="bp_pred_time", line=dict(color="green")), row=i+1, col=1)
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max() - 20, text="mse_test_time: " + str(mean_squared_error(y_test, y_pred)), showarrow=False, row=i+1, col=1)
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max() - 25, text="corr_test_time: " + str(np.corrcoef(y_test, y_pred)[0, 1]), showarrow=False,the row=i+1, col=1)
    # add corr and mse for time_learned model for train data
    y_train_pred = models[i].predict(x_train[['time']])
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max() - 30, text="mse_train_time: " + str(mean_squared_error(y_train, y_train_pred)), showarrow=False, row=i+1, col=1)
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max() - 35, text="corr_train_time: " + str(np.corrcoef(y_train, y_train_pred)[0, 1]), showarrow=False, row=i+1, col=1)
    #add predictions for train data
    fig.add_trace(go.Scatter(x=x_train['time'], y=y_train_pred, name="bp_pred_train_time", line=dict(color="green")), row=i+1, col=1)


fig.update_layout(height=5000, width=1000, title_text="bp over time for 5 patients")
fig.write_html("bp_over_time.html")
fig.show()

C:\Users\eythe\anaconda3\envs\LABSTAT\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning:

The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.



In [140]:
    # train the model
    models[i].fit(x, y)
    # predict the next 1000 values
    x_pred = pd.DataFrame({'time': np.linspace(x['time'].min(), x['time'].max() + 1000, 1000), 'drug': np.linspace(x['time'].min(), x['time'].max() + 1000, 1000)})
    x_pred['drug'] = x_pred['drug'].apply(lambda t: drug[(drug["patientunitstayid"] == pat) & (drug["infusionoffset"] <= t) & (drug["infusionoffset"] >= t - 5)]["drugamount"].to_numpy()[0] if drug[(drug["patientunitstayid"] == pat) & (drug["infusionoffset"] <= t) & (drug["infusionoffset"] >= t - 5)]["drugamount"].to_numpy().size > 0 else 0)
    y_pred, y_std = models[i].predict(x_pred, return_std=True)
    fig.add_trace(go.Scatter(x=x_pred["time"], y=y_pred, name="pred"), row=i+1, col=1)
    fig.add_trace(go.Scatter(x=x_pred["time"], y=y_pred + y_std, name="pred + std"), row=i+1, col=1)
    fig.add_trace(go.Scatter(x=x_pred["time"], y=y_pred - y_std, name="pred - std"), row=i+1, col=1)
    fig.add_trace(go.Scatter(x=x["time"], y=y, name="bp"), row=i+1, col=1)
    # calculate correlation between predicted and actual
    corr = np.corrcoef(y_pred, y)
    fig.update_layout(title_text=f"bp over time for 5 patients, corr={corr}")